# Control Systems 1, Coding Exercise 02: Jupyter Notebook on Introduction to System Classification and Properties
© 2024 ETH Zurich, Mark Benazet Castells, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli


This interactive notebook is designed to introduce fundamental concepts in control systems engineering. It covers the basics of system modeling, classification, and analysis, with a focus on building intuition through visual and interactive examples.

Authors:
- Mark Benazet Castells; mbenazet@ethz.ch
- Felix Muller; fmuller@ethz.ch

## Learning Objectives

After completing this notebook, you should be able to:

1. Understand input-output models of linear time-invariant (LTI) systems.
2. Write the model of an LTI system with A, B, C, D matrices.
3. Understand the concept of "state" to model a system.
4. Classify systems as static / dynamic, causal/non-causal, time-invariant/time-varying, linear/nonlinear.
5. Understand the notion of equilibrium points.
6. Approximate a non-linear system with a linear one.

# Setup

## Installing the required packages:
The following packages are required to run this notebook. If you haven't installed them yet, you can do so by running the following cell.

In [ ]:
%pip install numpy matplotlib scipy ipywidgets

## Import the packages:
The following cell imports the required packages. Run it before running the rest of the notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from ipywidgets import interact, interactive, fixed, widgets, FloatSlider, Dropdown
from IPython.display import display, Markdown, Latex, Image

# 1. Input-Output Models

An input-output model is a map Σ from an input signal u(t) to an output signal y(t):

$$y = \Sigma u$$

or more explicitly:

$$y(t) = (\Sigma u)(t), \quad \forall t \in T$$

This can be represented by a block diagram:

<div style="text-align:center;">
    <img src="./img/block_dgm.png" alt="Block Diagram" width="500">
</div>

Depending on how Σ affects u, we classify the system in different ways:
- Linear vs. Nonlinear
- Causal vs. Non-causal
- Static (memoryless) vs. Dynamic
- Time-invariant vs. Time-varying

# 2. Classification of Systems

## 1. Linear vs. Nonlinear Systems

A system is linear if it satisfies the superposition principle:

$\Sigma(a u_1 + b u_2) = a\Sigma(u_1) + b\Sigma(u_2)$

Or in other words:

$y(a u_1 + b u_2) = a y(u_1) + b y(u_2)$


Where Σ represents the system, u₁ and u₂ are input signals, and a and b are scalars.

Use the interactive plot below to check this property for different systems:

- Linear system: $y = u$
- Nonlinear system: $y = u^2$
- Nonlinear system: $y = \sin(u)$
- Nonlinear + Linear system: $y = \sin(u) + u$

Parameters:
- $a, b$: Scalars for the superposition principle

Adjust the parameters and observe:
- If the solid and dashed lines overlap for all inputs, the system is linear.
- If they diverge, the system is nonlinear.

In [ ]:
def system_response(system_type, u):
    if system_type == 'Linear':
        return u
    elif system_type == 'Nonlinear (Quadratic)':
        return u**2
    elif system_type == 'Nonlinear (Sine)':
        return np.sin(u)
    elif system_type == 'Nonlinear + Linear (Sine + Linear)':
        return np.sin(u) + u

def check_linearity(system_type, a, b):
    u1 = np.linspace(-2, 2, 200)
    u2 = np.linspace(-2, 2, 200)
    
    # Individual responses
    y1 = system_response(system_type, u1)
    y2 = system_response(system_type, u2)
    
    # Linearity check
    y_sum = system_response(system_type, a*u1 + b*u2)
    y_individual_sum = a*y1 + b*y2
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Linearity plot
    ax.plot(u1, y_sum, label='Σ(au₁ + bu₂)')
    ax.plot(u1, y_individual_sum, label='aΣ(u₁) + bΣ(u₂)', linestyle='--')
    ax.set_title('Linearity Check')
    ax.set_xlabel('Input')
    ax.set_ylabel('Output')
    ax.legend()
    ax.grid(True)
    
    plt.tight_layout()
    plt.show()

# Create interactive widgets
system_dropdown = Dropdown(options=['Linear', 'Nonlinear (Quadratic)', 'Nonlinear (Sine)', 'Nonlinear + Linear (Sine + Linear)'], 
                           value='Linear', description='System:')
a_slider = FloatSlider(min=-2, max=2, step=0.1, value=1, description='a:')
b_slider = FloatSlider(min=-2, max=2, step=0.1, value=1, description='b:')

# Create interactive plot
interactive_plot = interactive(check_linearity, system_type=system_dropdown, a=a_slider, b=b_slider)

# Display interactive plot
display(interactive_plot)

## 2. Causal vs Non-Causal Systems

1. **Causal System**: A system where the output at any time $t$ depends only on the values of the input on ($-\infty, t$]. 
   In other words, the system doesn't respond to future inputs.

2. **Non-Causal System**: A system where the output at time $t$ can depend on future inputs (after time $t$). 
   Such systems are not physically realizable in "the real world" as it would depend on future information.

## 3. Static vs. Dynamic

1. **Static System**: A system where the output at time $t$ depends only on the input at time $t$.
   In other words, the system has no memory of past inputs.

2. **Dynamic System**: A system where the output at time $t$ depends on the input at time $t$ and past inputs.

A good example of a static system is a **volume knob on a speaker**. The volume level is always controlled by the current position of the knob, without any consideration of previous settings. No matter how much the knob was turned earlier, the current volume depends only on the knob's current position.

An example of a dynamic system is the **longitudinal speed of a car**. A specific depression of the gas pedal does not correspond to one specific speed, because the car's speed depends on both how much the gas pedal is currently pressed and how it has been pressed over time. Other factors such as inertia and friction also contribute, meaning the speed evolves dynamically based on the system's history.

## 4. Time-Invariant vs. Time-Varying

1. **Time-Invariant System**: A system where the input-output relationship does not change over time.
   In other words, the system's properties do not change with time.

2. **Time-Varying System**: A system where the input-output relationship changes over time.

Imagine a perfect actuator in a control system:

1. Ideal Motor: 
   This actuator consistently produces the same output for a given input, regardless of when the input is applied. It's like a brand-new, high-quality motor that responds identically today, tomorrow, or a year from now.

2. Aging Motor:
   In reality, actuators may degrade over time due to wear and tear. An aging motor might produce less output for the same input as time passes. This change in behavior over time characterizes a time-varying system.

<div style="text-align:center;">
    <img src="./img/actuator_decay.png" alt="Block Diagram" width="800">
</div>

# 3. State-Space Models

When a system is causal, we can transform it from any form that might come out of the modeling techniques you saw last week to a state-space model. In general, a state space model looks like this:

$$\dot{x}(t) = f(t, x(t), u(t))$$
$$y(t) = g(t, x(t), u(t))$$
$$x(t_0) = x_0$$

Where:
- $x(t)$ is the state of the system.
- $u(t)$ is the input.
- $y(t)$ is the output.
- $x_0$ is the initial condition.

In this course, we will study only Linear Time-Invariant (LTI) systems. These systems are linear and time-invariant, which means they satisfy the superposition principle and their properties do not change over time. There are many useful techniques that we can't use for nonlinear systems that work great for LTI systems.

In the LTI case, the state-space model can be written as:

$$\dot{x}(t) = Ax(t) + Bu(t)$$
$$y(t) = Cx(t) + Du(t)$$
$$x(t_0) = x_0$$

Where:
- $A, B, C, D$ are matrices that define the system.

Most Systems are not LTI, but many systems can be approximated very well by LTI models. On the [graph above](#1-linear-vs-nonlinear-systems), you can see that $\Sigma(a u_1 + b u_2)$ and $a\Sigma(u_1) + b\Sigma(u_2)$ are very similar in some spots for the nonlinear systems, mainly the origin. We take advantage of that by **linearizing** nonlinear systems around these spots.

These spots, where a linearization is valid and useful, are called equilibrium points. For the general nonlinear state-space model, as seen model above, the equilibrium point $(x_e, u_e)$ is defined with:

$$f(x_e, u_e) = 0$$

This means that if $x_e$ is specified we can calculate $u_e$, and vice-versa. It's possible for a system to have multiple equilibrium points.

## 4. Linearization of a Nonlinear System

In this example, we explore the linearization of a nonlinear system around an equilibrium point. The system we analyze is the nonlinear function:

$$
f(x) = \sin(x)
$$

### Steps in the Process

EXPLAIN HERE HOW IT WORKS IN GENERAL

1. **Nonlinear System**: We define the system $f(x) = \sin(x)$. This system is nonlinear and periodic. It has equilibrium points where $f(x) = 0$, such as at $x = 0$, $x = \pi$, and $x = 2\pi$.

2. **Finding the Equilibrium Point**: In this case, we choose the equilibrium point $x = \pi$, where $\sin(\pi) = 0$.

3. **Linearization**: The linear approximation of a nonlinear system is given by the tangent line at the equilibrium point. To find this, we calculate the derivative of $f(x)$. For $f(x) = \sin(x)$, the derivative is:

$$
f'(x) = \cos(x)
$$

At $x = \pi$, the slope of the tangent line is:

$$
f'(\pi) = \cos(\pi) = -1
$$

Thus, the linear approximation of the system near $x = \pi$ is:

$$
y = -1 \cdot (x - \pi)
$$

### Visualization

In the visualisation you can see the nonlinear function as well as the linearisation in the point $x = \pi$. When you zoom in, meaning you see the area close to the linearization point, there is almost no difference between both functions. However, if you zoom out, you see that both functions begin to differ by a lot far from the equilibrium point.


In [ ]:
# Define the nonlinear system f(x) = sin(x)
def nonlinear_system(x):
    return np.sin(x)

# Calculate the linear approximation around the equilibrium point
def linear_approximation(x, equilibrium_point):
    # The derivative of sin(x) is cos(x), so the slope is cos(equilibrium_point)
    slope = np.cos(equilibrium_point)
    return slope * (x - equilibrium_point) + np.sin(equilibrium_point)

# Plot the nonlinear system and its linearization
def plot_system(zoom):
    # Generate x values based on zoom level
    x_vals = np.linspace(np.pi - zoom, np.pi + zoom, 400)
    
    # Nonlinear system output
    y_nonlinear = nonlinear_system(x_vals)
    
    # Linear approximation at equilibrium point x=π
    equilibrium_point = np.pi
    y_linear = linear_approximation(x_vals, equilibrium_point)
    
    # Plot the results
    plt.figure(figsize=(8, 6))
    
    # Plot the nonlinear system
    plt.plot(x_vals, y_nonlinear, label='Nonlinear System: $f(x) = \sin(x)$', color='blue')
    
    # Plot the linear approximation (tangent line)
    plt.plot(x_vals, y_linear, label=f'Linearization at $x=\pi$', linestyle='--', color='red')
    
    # Highlight the equilibrium point
    plt.scatter(equilibrium_point, nonlinear_system(equilibrium_point), color='black', zorder=5)
    plt.text(equilibrium_point + 0.1, np.sin(equilibrium_point), f'Equilibrium Point ($\pi$, 0)', fontsize=12)
    
    # Set plot details
    plt.title('Nonlinear System and Linearization around $x = \pi$')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    plt.grid(True)
    plt.xlim(np.pi - zoom, np.pi + zoom)
    plt.ylim(-zoom, zoom)
    plt.show()

# Add an interactive slider for zooming
interact(plot_system, zoom=(0.1, 4.0, 0.1))